In [1]:
import pandas as pd
import rasterio
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
import os
import pickle

In [2]:
with open(r'C:\Users\attic\HLS_Kelp\random_forest\rf.pickle', 'rb') as f:
    rf = pickle.load(f)
classified_path = r'C:\Users\attic\HLS_Kelp\imagery\rf_classified'
unclassified_path = r'C:\Users\attic\HLS_Kelp\imagery\rf_prepped'
files = os.listdir(unclassified_path)

In [3]:
for i, file in enumerate(files):
    if os.path.isfile(os.path.join(classified_path, f'{file}_kelp_classified.tif')):
        continue
    with rasterio.open(os.path.join(unclassified_path,file)) as img:
        file_img = img.read()
        width,height = img.shape
        img_data= file_img.reshape(file_img.shape[0], -1).T
        kelp_data = rf.predict(img_data)
        kelp_img = kelp_data.reshape(width, height)
        data_type = rasterio.uint8
        profile = {
            'driver': 'GTiff',
            'width': width,
            'height': height,
            'count': 1,  # one band
            'dtype': data_type,  # assuming binary mask, adjust dtype if needed
            'crs': img.crs,
            'transform': img.transform,
            'nodata': 0  # assuming no data is 0
        }
        # Write the land mask array to GeoTIFF
        with rasterio.open(os.path.join(classified_path, f'{file}_kelp_classified.tif'), 'w', **profile) as dst:
            dst.write(kelp_img.astype(data_type), 1)
        print(f'{i+1} / {len(files)}')
        

1 / 65
2 / 65
3 / 65
4 / 65
5 / 65
6 / 65
7 / 65
8 / 65
9 / 65
10 / 65
11 / 65
12 / 65
13 / 65
14 / 65
15 / 65
16 / 65
17 / 65
18 / 65
19 / 65
20 / 65
24 / 65
25 / 65
26 / 65
27 / 65
28 / 65
29 / 65
30 / 65
31 / 65
32 / 65
33 / 65
34 / 65
35 / 65
36 / 65
37 / 65
38 / 65
39 / 65
40 / 65
41 / 65
42 / 65
43 / 65
44 / 65
45 / 65
46 / 65
47 / 65
48 / 65
49 / 65
50 / 65
51 / 65
52 / 65
53 / 65
